# PoC: WeCookio multiagent Agno Agents

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Sequential workflow without delegation

### Installing Required Packages with uv

In [1]:
# Install required packages using uv
!uv add ipykernel agno==1.7.9 boto3 botocore pyyaml ipywidgets prompt-template sqlalchemy

Resolved 224 packages in 0.78ms
Audited 213 packages in 0.07ms


### Import Libraries and Load Configuration

In [2]:
import yaml
import logging
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.aws import AwsBedrock
from agno.workflow.v2.step import Step
from agno.workflow.v2.workflow import Workflow
from agno.run.v2.workflow import WorkflowRunResponse, WorkflowRunResponseEvent,WorkflowRunEvent
from agno.utils.pprint import pprint_run_response

print("Libraries imported successfully!")

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


Libraries imported successfully!


### Loading AWS Enviroment variables

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [4]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "agno/latest/"
# Load the integration configuration
with open(prefix+'config/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/config.yaml', 'r') as file:
    agno_config = yaml.safe_load(file)
with open(prefix+'config/agents-teams.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/tasks-teams.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(agno_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

/home/e2its/dev/weCookio/researches
Configuration files loaded successfully!
Integration config keys: ['aws_bedrock', 'database', 'api_endpoints']
Crew Orchestration keys: ['crew', 'agents', 'tasks', 'integration_config', 'model_assignment', 'execution', 'memory', 'input_schema', 'output_schema', 'cost_optimization', 'performance_metrics', 'quality_standards', 'monitoring', 'example_input', 'example_output']
Agents config keys: ['Culinary Coordinator', 'Dietary Compliance Specialist', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Quality Assurance Chef']
Tasks config keys: ['Enhance Recipe']


###  Model Selection based on complexity

In [5]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

High complexity model: bedrock/amazon.nova-premier-v1:0
Medium complexity model: bedrock/amazon.nova-pro-v1:0
Low complexity model: bedrock/amazon.nova-lite-v1:0
Ultra-Low complexity model: bedrock/amazon.nova-micro-v1:0


### Create Agents with Model-Specific Configuration

In [6]:
def create_agent_with_model(agent_config, model_config, **kwargs):
    """Create an agent with specific model configuration"""
    llm = AwsBedrock(
        id=model_config["model_id"].split("/")[-1],
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
    )

    args = {}
    args["model"]=llm
    args['role']=agent_config.get("role", None)
    args['goal']=agent_config.get("goal", None)
    args['instructions']=agent_config.get("prompt", None)
    args["description"]=agent_config.get("backstory", None)
    args["name"]=agent_config.get("name", None)
    
    return Agent(**args)

def print_agent_info(agent) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.name}\n"
        f"Agent Description: {agent.description}\n"
        f"Using model: {agent.model}\n"
        f"Role: {agent.role}\n"
        f"Goal: {agent.goal}\n"
        f"Instructions: {agent.instructions}\n"
    )
    print("\n")

In [ ]:
# Not used -Global metrics storage
def reset_execution_metrics():
    global execution_metrics
    execution_metrics = {
        'audio_tokens': [0],
        'cache_write_tokens': [0],
        'cached_tokens': [0],
        'completion_tokens': [0],
        'input_audio_tokens': [0],
        'input_tokens': [0],
        'output_audio_tokens': [0],
        'output_tokens': [0],
        'prompt_tokens': [0],
        'reasoning_tokens': [0],
        'time': [0],
        'total_tokens': [0]
    }
    return execution_metrics

def get_execution_metrics():
    return execution_metrics

def set_execution_metrics(kwargs):
    """Safely extract metrics from kwargs"""
    try:
        execution_metrics['audio_tokens'][0] += kwargs.get('audio_tokens')[0]
        execution_metrics['cache_write_tokens'][0] += kwargs.get('cache_write_tokens')[0]
        execution_metrics['cached_tokens'][0] += kwargs.get('cached_tokens')[0]
        execution_metrics['completion_tokens'][0] += kwargs.get('completion_tokens')[0]
        execution_metrics['input_audio_tokens'][0] += kwargs.get('input_audio_tokens')[0]
        execution_metrics['input_tokens'][0] += kwargs.get('input_tokens')[0]
        execution_metrics['output_audio_tokens'][0] += kwargs.get('output_audio_tokens')[0]
        execution_metrics['output_tokens'][0] += kwargs.get('output_tokens')[0]
        execution_metrics['prompt_tokens'][0] += kwargs.get('prompt_tokens')[0]
        execution_metrics['reasoning_tokens'][0] += kwargs.get('reasoning_tokens')[0]
        execution_metrics['time'][0] += kwargs.get('time')[0]
        execution_metrics['total_tokens'][0] += kwargs.get('total_tokens')[0]
        return execution_metrics
    except Exception as e:
        print(f"⚠️  Warning: Could not extract metrics: {e}")
    return execution_metrics

In [22]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent)


Created agent: Culinary Coordinator
Agent Description: Executive chef with 20+ years experience in dietary adaptations and team coordination
Using model: AwsBedrock(id='amazon.nova-lite-v1:0', name='AwsBedrock', provider='AwsBedrock', supports_native_structured_outputs=False, supports_json_schema_outputs=False, _tool_choice=None, system_prompt=None, instructions=None, tool_message_role='tool', assistant_message_role='assistant', aws_sso_auth=False, aws_region=None, aws_access_key_id=None, aws_secret_access_key=None, session=None, max_tokens=2000, temperature=0.4, top_p=0.9, stop_sequences=None, request_params=None, client=None, async_client=None, async_session=None)
Role: Master Chef & Project Manager
Goal: Orchestrate recipe adaptation ensuring maximum culinary excellence and strict dietary compliance
Instructions: You are the master chef for weCookio's culinary adaptation system.

**CRITICAL: Zero tolerance for dietary restriction violations - this is your top priority.**
**CRITICAL:

### Create a Flow

In [9]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Leche",
        "Huevo",
        "Ternera",
        "Maiz",
        "Gluten",
        "Arroz",
        "Ajo"
    ],
    "exclusions": ["Pimiento"],
    "preferences": [],
    "output_language": "spanish",
    "country": "Spain"
}

In [10]:
## Support functions
from prompt_template import PromptTemplate, InvalidTemplateKeysError, MissingTemplateValuesError, TemplateSerializationError

def variable_injection(task:dict, recipe_context:dict):
    try:
        prompt = PromptTemplate(name=task.get("name"), template=f'''{task.get("description")}''').to_string(**recipe_context)
    except MissingTemplateValuesError as e:
        print(f"Missing values: {e.missing_values}")  # {'name'}
    except InvalidTemplateKeysError as e:
        print(f"Invalid keys: {e.invalid_keys}")  # {'name'}
    except TemplateSerializationError as e:
        print(f"Serialization error: {e.message}")  # 'Invalid template string'
    except Exception as e:
        print(f"Unexpected error: {e}")  # 'Unexpected error'
    return prompt


In [11]:
tasks = {
    "enhance_recipe": {
        "description": f'''{variable_injection(tasks_config['Enhance Recipe'], recipe_request)}''',
        "dependencies": []
    }
}
print(tasks['enhance_recipe'].get("description")) 

Enhance the recipe with the following details:
## RECIPE CONTEXT
- **Recipe Name**: Canelones de ternera XXL al Pedro Ximenez
- **Servings**: 4
- **Dietary Restrictions**: ["Leche", "Huevo", "Ternera", "Maiz", "Gluten", "Arroz", "Ajo"], ["Pimiento"]
- **Preferences**: []
- **Target Country**: Spain
- **Output Language**: spanish

## TASK OBJECTIVE
Transform the requested recipe to be 100% compliant with dietary restrictions and adapted to preferences 
while maximizing culinary experience and preserving authentic flavor/texture characteristics and provide the expected output indicated bellow.

**Expected output:**
- Normalize the recipe and instructions to the target Spain
- Prepare a separate section with the expected changes in ingredients and their amounts to achieve the desired result.
- Return the detailed step by step instructions in the target Spain format and language spanish
- Validation status for Quality checks


In [23]:
# Sequential workflow processing
from agno.storage.sqlite import SqliteStorage

wecookio_workflow = Workflow(
    name="Enhacement of recipe to comply with dietary restrictions",
    description="Enhacement of recipe to comply with dietary restrictions",
    storage=SqliteStorage(
        table_name="wecookio_workflow",
        db_file="tmp/wecookio_wf_v2.db",
        mode="workflow_v2",
    ),
    steps=[culinary_coordinator_agent, ingredient_substitution_expert_agent, culinary_experience_optimizer_agent, quality_assurance_chef_agent],
)

def print_step_metrics(step_name, step_result):
    """Print metrics for an individual step"""
    if hasattr(step_result, 'metrics') and step_result.metrics:
        print(f"\n📊 {step_name} Step Metrics:")
        print(f"  • Input Tokens: {step_result.metrics.get('input_tokens', [0])[0]}")
        print(f"  • Output Tokens: {step_result.metrics.get('output_tokens', [0])[0]}")
        print(f"  • Total Tokens: {step_result.metrics.get('total_tokens', [0])[0]}")
        print(f"  • Time: {step_result.metrics.get('time', [0])[0]:.3f}s")
        print(f"  • Completion Tokens: {step_result.metrics.get('completion_tokens', [0])[0]}")
        print(f"  • Prompt Tokens: {step_result.metrics.get('prompt_tokens', [0])[0]}")
        
        # Update global metrics
        set_execution_metrics(step_result.metrics)
    else:
        print(f"\n⚠️  No metrics available for {step_name}")

def print_total_metrics():
    """Print accumulated total metrics"""
    metrics = get_execution_metrics()
    print(f"\n🏁 TOTAL EXECUTION METRICS:")
    print(f"  • Total Input Tokens: {metrics['input_tokens'][0]}")
    print(f"  • Total Output Tokens: {metrics['output_tokens'][0]}")
    print(f"  • Total Tokens: {metrics['total_tokens'][0]}")
    print(f"  • Total Time: {metrics['time'][0]:.3f}s")
    print(f"  • Total Completion Tokens: {metrics['completion_tokens'][0]}")
    print(f"  • Total Prompt Tokens: {metrics['prompt_tokens'][0]}")
    print(f"  • Audio Tokens: {metrics['audio_tokens'][0]}")
    print(f"  • Cached Tokens: {metrics['cached_tokens'][0]}")
    print(f"  • Cache Write Tokens: {metrics['cache_write_tokens'][0]}")


### Run the Flow

In [24]:
# Without streaming
try:
    # Reset metrics before execution
    reset_execution_metrics()
    print("🔄 Starting workflow execution...")
    
    result: WorkflowRunResponse = wecookio_workflow.run(
        message=tasks['enhance_recipe'].get("description"),
        markdown=True,
        stream=False,
        stream_intermediate_steps=False,
        debug_mode=True,
    )
    
    print("\n✅ Workflow execution completed!")
    
    # Print individual step metrics if available
    if hasattr(result, 'steps') and result.steps:
        print("\n📋 Individual Step Metrics:")
        for i, step in enumerate(result.steps):
            step_name = f"Step {i+1}"
            if hasattr(step, 'agent') and step.agent:
                step_name = step.agent.name if hasattr(step.agent, 'name') else step_name
            print_step_metrics(step_name, step)
    
    # Print total accumulated metrics
    print_total_metrics()
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

pprint_run_response(result, markdown=True)

🔄 Starting workflow execution...

✅ Workflow execution completed!

🏁 TOTAL EXECUTION METRICS:
  • Total Input Tokens: 0
  • Total Output Tokens: 0
  • Total Tokens: 0
  • Total Time: 0.000s
  • Total Completion Tokens: 0
  • Total Prompt Tokens: 0
  • Audio Tokens: 0
  • Cached Tokens: 0
  • Cache Write Tokens: 0


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                    Canelones de ternera XXL al Pedro Ximenez                                    │
│                                                                                                                 │
│                                         Ingredientes (para 4 porciones)                                         │
│                                                                                                                 │
│                                                    Canelones                                                    │
│                                                                                                                 │
│  • 200g de pasta de canelones sin gluten (Puedes hacerla en casa si prefieres)                                  │
│  • 1 taza de leche de almendra (en lugar de leche de vaca)                                                      │
│  • 1 huevo de codorniz (en lugar de huevo de gallina)                                                           │
│  • 1 taza de ternera molida (sin carne de cerdo ni pollo)                                                       │
│  • 1/2 taza de arroz sin gluten (en lugar de maíz)                                                              │
│  • 1/2 taza de caldo de verduras sin gluten                                                                     │
│  • 1/4 taza de Pedro Ximenez (vino dulce)                                                                       │
│  • 1/2 taza de pimiento rojo (sin gluten)                                                                       │
│  • 1/4 taza de aceite de oliva                                                                                  │
│  • Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│                                                     Relleno                                                     │
│                                                                                                                 │
│  • 1 taza de ternera molida (sin carne de cerdo ni pollo)                                                       │
│  • 1/2 taza de arroz sin gluten (en lugar de maíz)                                                              │
│  • 1/4 taza de caldo de verduras sin gluten                                                                     │
│  • 1/4 taza de Pedro Ximenez (vino dulce)                                                                       │
│  • 1/2 taza de pimiento rojo (sin gluten)                                                                       │
│  • 1/4 taza de aceite de oliva                                                                                  │
│  • Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│                                                      Salsa                                                      │
│                                                                                                                 │
│  • 1 taza de leche de almendra (en lugar de leche de vaca)                                                      │
│  • 1/4 taza de Pedro Ximenez (vino dulce)                                                                       │
│  • 1/4 taza de caldo de verduras sin gluten                                                                     │
│  • 1/4 taza de aceite de oliva                                                                                  │
│  • Sal y pimienta al gusto                            

In [ ]:
# With streaming
def process_streaming_workflow_debug():
    """Process workflow with debug information to understand event structure"""
    reset_execution_metrics()
    print("🔄 Starting debug streaming workflow execution...")
    
    try:
        result: Iterator[WorkflowRunResponseEvent] = wecookio_workflow.run(
            message=tasks['enhance_recipe'].get("description"),
            markdown=True,
            stream=True,
            stream_intermediate_steps=True,
        )
        
        for event in result:
            print(f"\n🔍 Event: {event.event}")
            print(f"📋 Event type: {type(event)}")
            print(f"🔧 Available attributes: {[attr for attr in dir(event) if not attr.startswith('_')]}")
            
            # Try to access common attributes safely
            if hasattr(event, 'step_name'):
                print(f"   Step name: {event.step_name}")
            if hasattr(event, 'step_id'):
                print(f"   Step ID: {event.step_id}")
            if hasattr(event, 'step'):
                print(f"   Step object: {event.step}")
                if hasattr(event.step, 'name'):
                    print(f"   Step name from object: {event.step.name}")
            
            if event.event == WorkflowRunEvent.workflow_started.value:
                print("🚀 Workflow started")
                
            elif event.event == WorkflowRunEvent.step_started.value:
                print("▶️  Step started")
                
            elif event.event == WorkflowRunEvent.step_completed.value:
                print("✅ Step completed")
                
                # Try to access result safely
                if hasattr(event, 'result'):
                    print(f"   Result object: {event.result}")
                    if hasattr(event.result, 'content'):
                        print(f"   Content: {event.result.content}")
                    if hasattr(event.result, 'metrics'):
                        print(f"   Metrics: {event.result.metrics}")
                else:
                    print("   No result attribute found")
                    
            elif event.event == WorkflowRunEvent.workflow_completed.value:
                print("�� Workflow completed!")
                print_total_metrics()
                
            print("-" * 50)

    except Exception as e:
        print(f"❌ Error during streaming: {e}")
        import traceback
        traceback.print_exc()

# Para usar streaming completo, descomenta esta línea:
#process_streaming_workflow_debug()
    